In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'Chapter2/wine.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
wine = pd.read_csv(file_content_stream)
wine.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [15]:
wine['Wine'].value_counts()

2    71
1    59
3    48
Name: Wine, dtype: int64

## Data Splitting

In [16]:
## Defining input and target variables
X = wine.drop(columns = 'Wine', axis = 1)
Y = wine['Wine']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Standardizing the data 
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## One-vs-all Classifier

In [20]:
## Building the multi-classifier (using RF) 
one_vs_all_RF = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test
one_vs_all_RF_pred = one_vs_all_RF.predict_proba(X_test)
one_vs_all_RF_pred = np.argmax(one_vs_all_RF_pred, axis = 1) + 1
one_vs_all_RF_pred

array([2, 3, 1, 1, 2, 3, 3, 3, 1, 1, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 1, 3,
       1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 3, 2, 3])

In [17]:
## Building the multi-classifier (using SVM) 
one_vs_all_svm = OneVsRestClassifier(estimator = SVC(kernel = 'rbf', probability = True)).fit(X_train, Y_train)

## Predicting on the test
one_vs_all_svm_pred = one_vs_all_svm.predict_proba(X_test)
one_vs_all_svm_pred = np.argmax(one_vs_all_svm_pred, axis = 1) + 1
one_vs_all_svm_pred

array([2, 3, 1, 1, 2, 3, 3, 3, 1, 1, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 3, 2, 3])

## One-vs-One Classifier

In [22]:
## Building the multi-classifier (using RF) 
one_vs_one_RF = OneVsOneClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test
one_vs_one_RF_pred = one_vs_one_RF.predict(X_test)
one_vs_one_RF_pred

array([2, 3, 1, 1, 2, 3, 3, 3, 1, 1, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 1, 3,
       1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 3, 2, 3])

In [23]:
## Building the multi-classifier (using SVM) 
one_vs_one_svm = OneVsOneClassifier(estimator = SVC(kernel = 'rbf', probability = True)).fit(X_train, Y_train)

## Predicting on the test
one_vs_one_svm_pred = one_vs_all_svm.predict_proba(X_test)
# one_vs_all_svm_pred = np.argmax(one_vs_one_svm_pred, axis = 1) + 1
one_vs_one_svm_pred

array([[1.36110978e-01, 8.53085491e-01, 1.08035304e-02],
       [2.41125977e-02, 4.44321565e-01, 5.31565837e-01],
       [9.79041910e-01, 1.26473550e-02, 8.31073516e-03],
       [7.94615604e-01, 1.82866024e-01, 2.25183722e-02],
       [6.50305669e-03, 9.71164634e-01, 2.23323096e-02],
       [1.03154790e-02, 3.64891595e-02, 9.53195361e-01],
       [1.93249068e-02, 6.34990880e-02, 9.17176005e-01],
       [8.26593164e-03, 4.41665705e-02, 9.47567498e-01],
       [9.18579875e-01, 6.22919919e-02, 1.91281332e-02],
       [9.81466709e-01, 9.09509504e-03, 9.43819623e-03],
       [9.93331225e-03, 4.31800378e-03, 9.85748684e-01],
       [2.17606298e-02, 2.27826493e-01, 7.50412877e-01],
       [2.48556081e-02, 2.42439334e-02, 9.50900459e-01],
       [6.47819525e-02, 9.09091056e-01, 2.61269915e-02],
       [1.06393597e-02, 5.43707231e-03, 9.83923568e-01],
       [1.51805428e-02, 9.75568667e-01, 9.25078980e-03],
       [6.28884554e-03, 9.92684902e-01, 1.02625201e-03],
       [5.35397867e-03, 9.88803